## 3. Data Enrichment - Understanding the "Why"

In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### 3.1. Load Datasets

In [11]:

db_user = 'root'
db_password = 'Kristine2004$'  
db_host = 'localhost'
db_name = 'banking_analysis'
connection_str = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_str)

df_losses = pd.read_csv('../data/banking_losses_for_tableau.csv', encoding='latin-1')
df_population = pd.read_csv('../data/states_by_population_2019.csv', encoding='latin-1')
df_gsdp = pd.read_csv('../data/states_by_GSDP_rs_billions_2018-23.csv', encoding='latin-1')
df_branches = pd.read_csv('../data/states_distribution_bankBranches_2023.csv', encoding='latin-1')
df_literacy = pd.read_csv('../data/states_by_literacy_percent_2023-24.csv', encoding='latin-1')
print("datasets loaded.")
print(df_losses)

datasets loaded.
                                       State  Loss_2018_19  Loss_2019_20  \
0                Andaman and Nicobar Islands          0.11          0.07   
1                             Andhra Pradesh          0.15          1.30   
2                          Arunachal Pradesh          0.01          0.05   
3                                      Assam          0.81          2.39   
4                                      Bihar          0.34          0.63   
5                                 Chandigarh          0.20          0.14   
6                               Chhattisgarh          0.25          0.30   
7   Dadra and Nagar Haveli and Daman and Diu          0.00          0.01   
8                                        Goa          0.36          0.31   
9                                    Gujarat          0.69          0.47   
10                                   Haryana         16.86         11.66   
11                          Himachal Pradesh          0.14          0.3

### 3.2. Data Cleaning and Standardization

In [12]:

df_population.rename(columns={'total_population': 'Population_Total', 'population_male': 'Population_Male', 'population_female': 'Population_Female'}, inplace=True)
df_gsdp.rename(columns={'State/Union Territory': 'State', '2018-19': 'GSDP_2018-19', '2019-20': 'GSDP_2019-20', '2020-21': 'GSDP_2020-21', '2021-22': 'GSDP_2021-22', '2022-23': 'GSDP_2022-23'}, inplace=True)
df_branches.rename(columns={'Branches_2023': 'Bank_Branches_Total'}, inplace=True)
df_literacy.rename(columns={'State/UT': 'State', 'Literacy Rate': 'Literacy_Rate_Percent'}, inplace=True)

all_dfs = [df_losses, df_population, df_gsdp, df_branches, df_literacy]
for df in all_dfs:
    if 'State' in df.columns:
        df['State'] = df['State'].str.upper().str.replace('&', 'AND').str.strip()

### 3.3. Loading into MySQL

In [13]:
df_losses.replace([np.inf, -np.inf], np.nan, inplace=True)

df_losses.to_sql('banking_losses', con=engine, if_exists='replace', index=False)
df_population.to_sql('state_population', con=engine, if_exists='replace', index=False)
df_gsdp.to_sql('state_gsdp', con=engine, if_exists='replace', index=False)
df_branches.to_sql('state_bank_branches', con=engine, if_exists='replace', index=False)
df_literacy.to_sql('state_literacy', con=engine, if_exists='replace', index=False)

print("All tables loaded into 'banking_analysis' database.")

All tables loaded into 'banking_analysis' database.
